In [1]:
import os 
import nibabel as nib
import numpy as np
import pandas as pd

# <span style="color:#FFCC80">In vivo Swiss mk.2 data processing </span> </br>
Compare and analyze if using optimized parameters improves or deteriorates the final local fields </br>

Keywords: </br>
* sw = slicewise
* pp = pipeline

In [ ]:
# We have 4 subjects inside
subjects = ['hc1', 'hc2', 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
# Now, from the root folder, select all the algorithms you want to compare
# They will be checked for all the subjects and all the measurements, if they don't exist they are skipped
# Therefore try to have the same folder names for any tests you want to compare
algo_comparing = []
rows = []


In [2]:
root_dir = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing"

In [3]:
# Lets create a "edge mask" of the spinal cord for hc2m1 for image showing in abstract and paper
sc_mask_hc2m1_img = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom1_sc_msk.nii.gz")
sc_mask_hc2m1_data = sc_mask_hc2m1_img.get_fdata()

In [ ]:
from scipy.ndimage import binary_erosion, binary_dilation
edges_hc2m1 = binary_dilation(sc_mask_hc2m1_data)  & ~binary_erosion(sc_mask_hc2m1_data)
edges_sc_msk_hc2m1_img = nib.Nifti1Image(edges_hc2m1.astype(np.uint8) , affine=sc_mask_hc2m1_img.affine)

In [ ]:
nib.save(edges_sc_msk_hc2m1_img,r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2_m1_custiom_sc_edges_msk.nii.gz" )

# <span style="color:#ff007f">Translation of in-sillico results to in-vivo </span> </br>

## <span style="color:#ffaa00">FANSI in-vivo checks</span> </br>
We wil use subject hc2 measurement 1 because it had the best perforiming local field - with optimized pdf


In [ ]:
# # We have 4 subjects inside
subjects = ['hc2']
measurements = ['m1']
sanitity_check_dir =  r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\chi_map\fansi\all_bgfr_to_custom_fansi"

# Put opt_pdf_test_tolerance after tansi and then lambda test folder names:
#'opt_pdf_test_tolerance/def_tol', 'opt_pdf_test_tolerance/higher_tol', 'opt_pdf_test_tolerance/lower_tol', from the tolerance tests, all the other parameters are default
# the best was the one with lower tolerance at 0.761 but default got 0.760 so neglibible difference
# Now testing the lambda, fixing tolerance to the default, fixing mu2 to 0.5 and gradient consistency to 0.1
#-#
# II. Lambda tests, put this after fansi: opt_pdf_to_opt_fansi/I_lambda_tests
# Folder names: "test1_lambda_0_00033", "test2_lambda_def", "test3_lambda_0_001"
#-#
# III mu1 tests, put opt_pdf_to_opt_fansi\II_mu1_tests after fansi
# Foldernames: ['test1_lower_mu1', 'test2_higher_mu1']
#-#
# IV mu2 put opt_pdf_to_opt_fansi\III_mu2_tests after fansi
# Foldernames: ['test1_0_3', 'test2_0_7', 'test3_0_1']
#-#
# V Extra tests, put opt_pdf_to_opt_fansi\extra_tests after fansi
# Foldernames: ['test1', 'test2', 'test3', 'test4', 'test5']
#-#
# Lastly to compare with LBV, default BGFR algos and more
# Put all_bgfr_to_custom1_fansi after fansi, this "custom1" fansi uses lambda = 0.001 , mu2 = 1 and mu1 = 1
# Custom 1: Using opt pdf into custom 1 fansi 
# Custom 2: default lbv into fansi with default params
# Custom 3: using default pdf into fansi with default params
# Custom 4: opt lbv into custom 1 fansi
# Custom 5: opt resharp into custom 1 fansi
# Results where: 
# Sorting by final metric: 4 > 5 > 1 > 3 > 2
# Sorting by delta_chi wm-gm: 4 > 3 > 5 > 1 > 2

test_folders = ['custom1', 'custom2', 'custom3', 'custom4', 'custom5']
fansi_testing_chi_map_rows = []

In [126]:
# Now lets collect rows for the testin folders
fansi_testing_chi_map_rows = []
for algo in test_folders:
            
            # Firsst, define the path pointers
            
            chi_map_path = os.path.join(sanitity_check_dir, algo,'Sepia_Chimap.nii.gz')
            if chi_map_path is None or not os.path.exists(chi_map_path):
                print(f"Skipping {chi_map_path} as it does not exist.")
                continue

            gm_msk_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_gm_msk.nii.gz"
            wm_msk_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz"

            # Load data
            chi_map_img = nib.load(chi_map_path)
            chi_map_data = chi_map_img.get_fdata()

            gm_vals = chi_map_data[(gm_mask == 1) & (chi_map_data != 0)].ravel()
            wm_vals = chi_map_data[(wm_mask == 1) & (chi_map_data != 0)].ravel()

            gm_mask = nib.load(gm_msk_path).get_fdata()
            total_vox_gm = np.sum(gm_mask==1) 
            wm_mask = nib.load(wm_msk_path).get_fdata()
            total_vox_wm = np.sum(wm_mask==1)

            # Compute metrics for GM and WM
            gm_mean = np.mean(chi_map_data[gm_mask==1])
            gm_std = np.std(chi_map_data[gm_mask==1])
            n_gm = gm_mean.size
            wm_mean = np.mean(chi_map_data[wm_mask==1])
            wm_std = np.std(chi_map_data[wm_mask==1])
            n_wm = wm_mean.size

            # Compute how many voxels are in the mask
            gm_nonzero_vox = np.sum(chi_map_data[gm_mask==1] != 0)
            wm_nonzero_vox = np.sum(chi_map_data[wm_mask==1] != 0)

            # Compute the contrast metric
            # We want to maximize the contrast between GM and WM
            # WM mean should be negative, if it is positive the contrast will be lower this way
            contrast = np.abs(gm_mean - wm_mean)

            normalizer_denominator_2 = np.sqrt(gm_std**2 + wm_std**2) 
           
            raw_metric = contrast / normalizer_denominator_2 if normalizer_denominator_2 != 0 else 0 # Just in case that the std is 0 - to avoid division by zero

            # Now we penalize if the algo eroded the mask
            gm_penality = gm_nonzero_vox / total_vox_gm 
            wm_penality = wm_nonzero_vox / total_vox_wm

            # Final metric:
            final_metric = raw_metric * gm_penality * wm_penality

            # Now collect row and add to data frame
            fansi_testing_chi_map_rows.append({
                    
                'subject': 'hc2',
                'measurement': 'm1',
                'test_folder': algo,

                'mean_gm': gm_mean,
                'std_gm': gm_std,
                'total_vox_gm': total_vox_gm,
                'nonzero_vox_gm': gm_nonzero_vox,
                'gm_penality': gm_penality,

                'mean_wm': wm_mean,
                'std_wm': wm_std,
                'total_vox_wm': total_vox_wm,
                'nonzero_vox_wm': wm_nonzero_vox,
                'wm_penality': wm_penality,
    
                'contrast_factor': contrast,
                'pooled_std_denominator': normalizer_denominator_2,
                #'raw_metric': raw_metric,
                
                'final_metric': final_metric
            })


# Create the data frame
sanity_check_df = pd.DataFrame(fansi_testing_chi_map_rows)
# No need to save because this is just a sanity check

In [128]:
sorted_sanity_check_df = sanity_check_df.sort_values(by='contrast_factor', ascending=False)
sorted_sanity_check_df

,subject,measurement,test_folder,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,gm_penality,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,wm_penality,contrast_factor,pooled_std_denominator,final_metric
3,hc2,m1,custom4,0.011256,0.011829,477,476,0.997904,-0.002384,0.007704,2252,2252,1.0,0.013641,0.014116,0.964301
2,hc2,m1,custom3,0.009873,0.011427,477,476,0.997904,-0.002091,0.011254,2252,2252,1.0,0.011965,0.016038,0.744444
4,hc2,m1,custom5,0.008833,0.010036,477,476,0.997904,-0.001871,0.006516,2252,2252,1.0,0.010704,0.011966,0.892686
0,hc2,m1,custom1,0.008399,0.010199,477,476,0.997904,-0.001779,0.007020,2252,2252,1.0,0.010178,0.012381,0.820346
1,hc2,m1,custom2,0.004101,0.008378,477,476,0.997904,-0.000869,0.003675,2252,2252,1.0,0.004970,0.009149,0.542086


## <span style="color:#6aff0e">Comparing DI with opt-PDF LF in-vivo checks</span> </br>
We wil use subject hc2 measurement 1 because it had the best perforiming local field - with optimized pdf


In [134]:
# # We have 4 subjects inside
subjects = ['hc2']
measurements = ['m1']
sanitity_check_dir =  r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\chi_map\opt_pdf_to_all_DI"

# Put opt_pdf_test_tolerance after tansi and then lambda test folder names:
#-#
# Lastly to compare with LBV, default BGFR algos and more
# Put all_bgfr_to_custom1_fansi after fansi, this "custom1" fansi uses 0.001 lambda, mu2 = 1 and mu1 = 1
# Custom 1: opt_TKD
# Custom 2: def_TKD
# Custom 3: opt_NDI
# Custom 4: def_NDI
# Custom 5: opt_iLSQR
# Custom 6: def_iLSQR
# Custom 7: test_lower_TKD
# Custom 8: opt_fansi
# Custom 9: def_fansi

test_folders = ['opt_TKD', 'def_TKD', 'opt_NDI', 'def_NDI', 'opt_iLSQR', 'def_iLSQR', 'test_lower_TKD', 'opt_fansi', 'def_fansi']
testing_chi_map_rows = []

In [135]:
# Call the function and create CSV
# Now lets collect rows for the testin folders
testing_chi_map_rows = []
for algo in test_folders:
            
            # Firsst, define the path pointers
            
            chi_map_path = os.path.join(sanitity_check_dir, algo,'Sepia_Chimap.nii.gz')
            if chi_map_path is None or not os.path.exists(chi_map_path):
                print(f"Skipping {chi_map_path} as it does not exist.")
                continue

            gm_msk_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_gm_msk.nii.gz"
            wm_msk_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz"

            # Load data
            chi_map_img = nib.load(chi_map_path)
            chi_map_data = chi_map_img.get_fdata()

            gm_vals = chi_map_data[(gm_mask == 1) & (chi_map_data != 0)].ravel()
            wm_vals = chi_map_data[(wm_mask == 1) & (chi_map_data != 0)].ravel()

            gm_mask = nib.load(gm_msk_path).get_fdata()
            total_vox_gm = np.sum(gm_mask==1) 
            wm_mask = nib.load(wm_msk_path).get_fdata()
            total_vox_wm = np.sum(wm_mask==1)

            # Compute metrics for GM and WM
            gm_mean = np.mean(chi_map_data[gm_mask==1])
            gm_std = np.std(chi_map_data[gm_mask==1])
            n_gm = gm_mean.size
            wm_mean = np.mean(chi_map_data[wm_mask==1])
            wm_std = np.std(chi_map_data[wm_mask==1])
            n_wm = wm_mean.size

            # Compute how many voxels are in the mask
            gm_nonzero_vox = np.sum(chi_map_data[gm_mask==1] != 0)
            wm_nonzero_vox = np.sum(chi_map_data[wm_mask==1] != 0)

            # Compute the contrast metric
            # We want to maximize the contrast between GM and WM
            # WM mean should be negative, if it is positive the contrast will be lower this way
            contrast = np.abs(gm_mean - wm_mean)

            normalizer_denominator_2 = np.sqrt(gm_std**2 + wm_std**2) 
           
            raw_metric = contrast / normalizer_denominator_2 if normalizer_denominator_2 != 0 else 0 # Just in case that the std is 0 - to avoid division by zero

            # Now we penalize if the algo eroded the mask
            gm_penality = gm_nonzero_vox / total_vox_gm 
            wm_penality = wm_nonzero_vox / total_vox_wm

            # Final metric:
            final_metric = raw_metric * gm_penality * wm_penality

            # Now collect row and add to data frame
            testing_chi_map_rows.append({
                    
                'subject': 'hc2',
                'measurement': 'm1',
                'test_folder': algo,

                'mean_gm': gm_mean,
                'std_gm': gm_std,
                'total_vox_gm': total_vox_gm,
                'nonzero_vox_gm': gm_nonzero_vox,
                'gm_penality': gm_penality,

                'mean_wm': wm_mean,
                'std_wm': wm_std,
                'total_vox_wm': total_vox_wm,
                'nonzero_vox_wm': wm_nonzero_vox,
                'wm_penality': wm_penality,
    
                'contrast_factor': contrast,
                'pooled_std_denominator': normalizer_denominator_2,
                #'raw_metric': raw_metric,
                
                'final_metric': final_metric
            })


# Create the data frame
sanity_check_df = pd.DataFrame(testing_chi_map_rows)
# No need to save because this is just a sanity check


In [136]:
contrast_sorted_sanity_check_df = sanity_check_df.sort_values(by='contrast_factor', ascending=False)
contrast_sorted_sanity_check_df

,subject,measurement,test_folder,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,gm_penality,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,wm_penality,contrast_factor,pooled_std_denominator,final_metric
4,hc2,m1,opt_iLSQR,0.008794,0.011262,477,476,0.997904,-0.001863,0.008698,2252,2252,1.0,0.010657,0.014230,0.747311
8,hc2,m1,def_fansi,0.008738,0.010812,477,476,0.997904,-0.001851,0.008723,2252,2252,1.0,0.010589,0.013892,0.760650
1,hc2,m1,def_TKD,0.008723,0.011113,477,476,0.997904,-0.001848,0.008422,2252,2252,1.0,0.010570,0.013944,0.756474
3,hc2,m1,def_NDI,0.008693,0.011209,477,476,0.997904,-0.001841,0.008638,2252,2252,1.0,0.010534,0.014151,0.742822
2,hc2,m1,opt_NDI,0.008690,0.011223,477,476,0.997904,-0.001841,0.008658,2252,2252,1.0,0.010531,0.014174,0.741438
0,hc2,m1,opt_TKD,0.008660,0.011594,477,476,0.997904,-0.001834,0.008649,2252,2252,1.0,0.010494,0.014465,0.723982
6,hc2,m1,test_lower_TKD,0.008650,0.011828,477,476,0.997904,-0.001832,0.008722,2252,2252,1.0,0.010482,0.014696,0.711779
7,hc2,m1,opt_fansi,0.008399,0.010199,477,476,0.997904,-0.001779,0.007020,2252,2252,1.0,0.010178,0.012381,0.820346
5,hc2,m1,def_iLSQR,0.007028,0.007471,477,476,0.997904,-0.001489,0.005633,2252,2252,1.0,0.008516,0.009357,0.908284


In [137]:
noise_sorted_sanity_check_df = sanity_check_df.sort_values(by='pooled_std_denominator', ascending=True)
noise_sorted_sanity_check_df

,subject,measurement,test_folder,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,gm_penality,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,wm_penality,contrast_factor,pooled_std_denominator,final_metric
5,hc2,m1,def_iLSQR,0.007028,0.007471,477,476,0.997904,-0.001489,0.005633,2252,2252,1.0,0.008516,0.009357,0.908284
7,hc2,m1,opt_fansi,0.008399,0.010199,477,476,0.997904,-0.001779,0.007020,2252,2252,1.0,0.010178,0.012381,0.820346
8,hc2,m1,def_fansi,0.008738,0.010812,477,476,0.997904,-0.001851,0.008723,2252,2252,1.0,0.010589,0.013892,0.760650
1,hc2,m1,def_TKD,0.008723,0.011113,477,476,0.997904,-0.001848,0.008422,2252,2252,1.0,0.010570,0.013944,0.756474
3,hc2,m1,def_NDI,0.008693,0.011209,477,476,0.997904,-0.001841,0.008638,2252,2252,1.0,0.010534,0.014151,0.742822
2,hc2,m1,opt_NDI,0.008690,0.011223,477,476,0.997904,-0.001841,0.008658,2252,2252,1.0,0.010531,0.014174,0.741438
4,hc2,m1,opt_iLSQR,0.008794,0.011262,477,476,0.997904,-0.001863,0.008698,2252,2252,1.0,0.010657,0.014230,0.747311
0,hc2,m1,opt_TKD,0.008660,0.011594,477,476,0.997904,-0.001834,0.008649,2252,2252,1.0,0.010494,0.014465,0.723982
6,hc2,m1,test_lower_TKD,0.008650,0.011828,477,476,0.997904,-0.001832,0.008722,2252,2252,1.0,0.010482,0.014696,0.711779


In [138]:
final_metric_sorted_sanity_check_df = sanity_check_df.sort_values(by='final_metric', ascending=False)
final_metric_sorted_sanity_check_df

,subject,measurement,test_folder,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,gm_penality,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,wm_penality,contrast_factor,pooled_std_denominator,final_metric
5,hc2,m1,def_iLSQR,0.007028,0.007471,477,476,0.997904,-0.001489,0.005633,2252,2252,1.0,0.008516,0.009357,0.908284
7,hc2,m1,opt_fansi,0.008399,0.010199,477,476,0.997904,-0.001779,0.007020,2252,2252,1.0,0.010178,0.012381,0.820346
8,hc2,m1,def_fansi,0.008738,0.010812,477,476,0.997904,-0.001851,0.008723,2252,2252,1.0,0.010589,0.013892,0.760650
1,hc2,m1,def_TKD,0.008723,0.011113,477,476,0.997904,-0.001848,0.008422,2252,2252,1.0,0.010570,0.013944,0.756474
4,hc2,m1,opt_iLSQR,0.008794,0.011262,477,476,0.997904,-0.001863,0.008698,2252,2252,1.0,0.010657,0.014230,0.747311
3,hc2,m1,def_NDI,0.008693,0.011209,477,476,0.997904,-0.001841,0.008638,2252,2252,1.0,0.010534,0.014151,0.742822
2,hc2,m1,opt_NDI,0.008690,0.011223,477,476,0.997904,-0.001841,0.008658,2252,2252,1.0,0.010531,0.014174,0.741438
0,hc2,m1,opt_TKD,0.008660,0.011594,477,476,0.997904,-0.001834,0.008649,2252,2252,1.0,0.010494,0.014465,0.723982
6,hc2,m1,test_lower_TKD,0.008650,0.011828,477,476,0.997904,-0.001832,0.008722,2252,2252,1.0,0.010482,0.014696,0.711779


# Comparing for all subjects and all measurements (not ready yet)

In [ ]:
# Now, lets see which hc1 chi map is better

Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m2\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m2\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m3\local_field\opt_resharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m3\local_field\opt_sharp/dilated_mask\Sepia_localfield.nii.gz as it does not exist.
Skipping E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m4\local_field\opt_resharp/dilated_mask\Sepia_localfield

# <span style="color:#16D8E6">In vivo QSM metric analysis</span> </br>
If you've done everything following instructions, under each dub/meas there should be: </br>
- hcxmx_opt_pdf_opt_tkd_gm_metrics.csv
- hcxmx_opt_pdf_opt_tkd_wm_metrics.csv
 
But first let's see what's inside one of them as example using hc2m1



In [21]:
import algo_comp_from_folder 
import importlib
importlib.reload(algo_comp_from_folder)
from algo_comp_from_folder import calculate_spine_avg_qsm_metrics

In [19]:
%load_ext autoreload
%autoreload 2
import algo_comp_from_folder

In [27]:
hc2m1_qsm_gm_metrics = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2m1_opt_pdf_opt_tkd_gm_metrics.csv")
hc2m1_qsm_wm_metrics = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2m1_opt_pdf_opt_tkd_wm_metrics.csv")

In [5]:
# This metric extraction was done per level so we have slicewise averages, lets show only WA and STD
hc2m1_qsm_gm_metrics.head()

,Timestamp,SCT Version,Filename,Slice (I->S),VertLevel,DistancePMJ,Label,Size [vox],WA(),STD()
0,2025-10-23 15:33:20,7.0,E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processi...,0,NaN,NaN,custom_hc2_m1_gm_msk,0,NaN,NaN
1,2025-10-23 15:33:20,7.0,E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processi...,1,NaN,NaN,custom_hc2_m1_gm_msk,0,NaN,NaN
2,2025-10-23 15:33:20,7.0,E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processi...,2,NaN,NaN,custom_hc2_m1_gm_msk,44,0.008108,0.011192
3,2025-10-23 15:33:20,7.0,E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processi...,3,NaN,NaN,custom_hc2_m1_gm_msk,50,0.009505,0.011644
4,2025-10-23 15:33:20,7.0,E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processi...,4,NaN,NaN,custom_hc2_m1_gm_msk,51,0.011643,0.015838


In [28]:
hc2m1_qsm_gm_metrics['WA()'][::-1]

15         NaN
14         NaN
13         NaN
12    0.008348
11    0.011650
10    0.012782
9     0.012251
8     0.011531
7     0.005974
6     0.010162
5     0.015772
4     0.011643
3     0.009505
2     0.008108
1          NaN
0          NaN
Name: WA(), dtype: float64

In [29]:
hc2m1_qsm_wm_metrics['WA()'][::-1]

15         NaN
14         NaN
13         NaN
12   -0.001249
11   -0.001759
10   -0.002283
9    -0.001560
8    -0.002778
7    -0.000011
6    -0.000633
5    -0.003176
4    -0.003382
3    -0.000584
2    -0.002501
1          NaN
0          NaN
Name: WA(), dtype: float64

In [33]:
hc1m1_qsm_gm_metrics = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\hc1m1_opt_pdf_opt_tkd_gm_metrics.csv")
hc1m1_qsm_wm_metrics = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\hc1m1_opt_pdf_opt_tkd_wm_metrics.csv")
hc1m1_qsm_gm_avg = np.mean(hc1m1_qsm_gm_metrics['WA()'])
hc1m1_qsm_wm_avg = np.mean(hc1m1_qsm_wm_metrics['WA()'])
print("The average GM value for hc1 m1 is:", hc1m1_qsm_gm_avg)
print("The average WM value for hc1 m1 is:", hc1m1_qsm_wm_avg)

The average GM value for hc1 m1 is: 0.010627142263636365
The average WM value for hc1 m1 is: -0.0021232499163636367


In [34]:
# We can add them to calcualte average gm value along the spine:
hc2m1_qsm_gm_avg = np.mean(hc2m1_qsm_gm_metrics['WA()'])
hc2m1_qsm_wm_avg = np.mean(hc2m1_qsm_wm_metrics['WA()'])
print("The average GM value for hc2 m1 is:", hc2m1_qsm_gm_avg)
print("The average WM value for hc2 m1 is:", hc2m1_qsm_wm_avg)

The average GM value for hc2 m1 is: 0.010702331027272728
The average WM value for hc2 m1 is: -0.0018104496176363637


In [ ]:
# Average between subjects:
total_dub_gm_avg = (hc1m1_qsm_gm_avg + hc2m1_qsm_gm_avg) / 2
total_dub_wm_avg = (hc1m1_qsm_wm_avg + hc2m1_qsm_wm_avg) / 2
print("The average GM value for both hc1 and hc2, m1 is:", total_dub_gm_avg)
print("The average WM value for both hc1 and hc2, m1 is:", total_dub_wm_avg)


The average GM value for both hc1 and hc2 m1 is: 0.010664736645454547
The average WM value for both hc1 and hc2 m1 is: -0.001966849767


In [24]:
root = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing"
dubs = ['hc1', 'hc2']
measurements = ['m1', 'm2', 'm3', 'm4']
algo_identifier=  "opt_pdf_opt_tkd"

spine_gm_avg, spine_wm_avg = calculate_spine_avg_qsm_metrics(root, dubs, measurements, algo_identifier)

In [25]:
total_dub_wm_avg = np.mean(spine_wm_avg)
total_dub_gm_avg = np.mean(spine_gm_avg)

In [26]:
print("The average GM value across all subjects and measurements is:", total_dub_gm_avg)
print("The average WM value across all subjects and measurements is:", total_dub_wm_avg)

The average GM value across all subjects and measurements is: 0.008828875882325758
The average WM value across all subjects and measurements is: -0.0016812606870454546


### <span style="color:#ffaa7f">STD calculation</span> </br>

In [45]:
# Now lets collect information about the standar deviation
slice_stds = hc1m1_qsm_gm_metrics['STD()'].to_numpy()
slice_means = hc1m1_qsm_gm_metrics['WA()'].to_numpy()
slice_voxels = hc1m1_qsm_gm_metrics['Size [vox]'].to_numpy()

In [46]:
mask = ~np.isnan(slice_stds) & ~np.isnan(slice_means) & (slice_voxels > 0)

In [47]:
slice_stds = slice_stds[mask]
slice_means = slice_means[mask]
slice_voxels = slice_voxels[mask]

In [48]:
pooled_var = np.sum((slice_stds**2 * (slice_voxels - 1) + slice_voxels * (slice_means - np.average(slice_means, weights=slice_voxels))**2)) / np.sum(slice_voxels)
pooled_std = np.sqrt(pooled_var)
print("The pooled standard deviation across the spine for hc1 m1 GM is:", pooled_std)

The pooled standard deviation across the spine for hc1 m1 GM is: 0.016852180300888503


In [49]:
hc1m1_slice_gm_std = hc1m1_qsm_gm_metrics['STD()']
hc1m1_all_slice_gm_std = np.mean(hc1m1_slice_gm_std)

hc2m1_slice_gm_std = hc2m1_qsm_gm_metrics['STD()']
hc2m1_all_slice_gm_std = np.mean(hc2m1_slice_gm_std)

hc1m1_slice_wm_std = hc1m1_qsm_wm_metrics['STD()']
hc1m1_all_slice_wm_std = np.mean(hc1m1_slice_wm_std)

hc2m1_slice_wm_std = hc2m1_qsm_wm_metrics['STD()']
hc2m1_all_slice_wm_std = np.mean(hc2m1_slice_wm_std)

In [50]:
# GM across subjects (approximate)
all_gm_std = np.mean([hc1m1_all_slice_gm_std, hc2m1_all_slice_gm_std])

# WM across subjects (approximate)
all_wm_std = np.mean([hc1m1_all_slice_wm_std, hc2m1_all_slice_wm_std])


In [51]:
print("Standard deviation across subjects for GM is:", all_gm_std)
print("Standard deviation across subjects for WM is:", all_wm_std)

Standard deviation across subjects for GM is: 0.01289265775893854
Standard deviation across subjects for WM is: 0.012257995202797807


# chi_003

In [58]:
import pydicom
import glob

dicoms = sorted(glob.glob(r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\raw_data\06-3D_mgre_GRAPPA_5TEs"))
for d in dicoms[:5]:
    dcm = pydicom.dcmread(d)
    print(dcm.EchoTime)



PermissionError: [Errno 13] Permission denied: 'E:\\msc_data\\sc_qsm\\neuropoly_data\\chi_003\\raw_data\\06-3D_mgre_GRAPPA_5TEs'